In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import pandas as pd
import numpy as np
import random
from tqdm import tqdm

import torch

from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pathlib import Path

from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig

import logging
from pathlib import Path

pd.set_option('display.max_rows', 1000)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
EMB_FILE = Path('../data/embeddings/gg2.2024.09.backbone.full-length.DNABERTS.npy')
CSV_FILE = Path('../data/gg2.2024.09.backbone.full-length.csv.gz')

In [10]:
def load_data() -> tuple[np.ndarray, pd.DataFrame]:
    """
    Load embeddings and CSV data files.

    Returns:
        tuple: A tuple containing embeddings (numpy array) and DataFrame (pandas).
    """
    try:
        # Check if files exist
        if not EMB_FILE.is_file():
            raise FileNotFoundError(f"Embedding file not found: {EMB_FILE}")
        if not CSV_FILE.is_file():
            raise FileNotFoundError(f"CSV file not found: {CSV_FILE}")
        
        logging.info(f"Loading embeddings from {EMB_FILE}")
        embeddings = np.load(EMB_FILE)
        
        logging.info(f"Loading DataFrame from {CSV_FILE}")
        df = pd.read_csv(CSV_FILE, sep='\t')
        
        return embeddings, df

    except Exception as e:
        logging.error(f"Error while loading data: {e}")
        raise

In [ ]:
def load_model_from_hf(model_name: str, maskedlm: bool = False, causallm: bool = False):
    """
    Loads a model and tokenizer from Hugging Face.

    Args:
        model_name (str): The model name or path in Hugging Face Hub.

    Returns:
        tuple: A tuple containing the loaded model and tokenizer.

    Raises:
        ValueError: If the model name is invalid or loading fails.
    """
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )
        if maskedlm:
            model = AutoModelForMaskedLM.from_pretrained(
                model_name,
                trust_remote_code=True,
                #device_map={"": 0}
            )
        elif causallm:
            model_config = AutoConfig.from_pretrained(model_name, trust_remote_code=True, revision="1.1_fix")
            model_config.use_cache = True
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                config=model_config,
                trust_remote_code=True,
                revision="1.1_fix",
                torch_dtype=torch.float16,
            )
        else:
            model = AutoModel.from_pretrained(
                model_name,
                trust_remote_code=True,
                #device_map={"": 0}
            )
        return model, tokenizer
    except Exception as e:
        raise ValueError(f"Failed to load model/tokenizer: {e}")

In [11]:
try:
    embeddings, df = load_data()
    logging.info("Data loaded successfully.")
except Exception as e:
    logging.error("Failed to load data.")

2025-02-27 20:31:36,753 - INFO - Loading embeddings from ..\..\..\Data\Genomes\Greengenes2\2024.09\embeddings\df.2024.09.backbone.full-length.DNABERT2.npy
2025-02-27 20:31:38,531 - INFO - Loading DataFrame from ..\..\..\Data\Genomes\Greengenes2\2024.09\df.2024.09.backbone.full-length.csv.gz
2025-02-27 20:31:52,019 - INFO - Data loaded successfully.


# DNABERT-2

In [ ]:
MODEL_ID = "zhihan1996/DNABERT-2-117M"
model, tokenizer = load_model_from_hf(model_name=MODEL_ID)

Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compare_reduced_length(seq: str = None, orig_emb: np.ndarray = None, n: int = 100):
    results = [1]
    steps = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    for i in steps:
        seq_short = seq[i:-i]
        inputs = tokenizer(seq_short, return_tensors = 'pt')["input_ids"].to("cuda")
        hidden_states = model(inputs)[0]
        embedding_mean = torch.mean(hidden_states[0], dim=0).cpu().reshape(1, 768).detach().numpy()
        cos_sim = cosine_similarity(orig_emb.reshape(1, -1), embedding_mean.reshape(1, -1))[0][0]
        results.append(cos_sim)
    return results

In [15]:
n_samples = 1000
n = 100
all_results = []

for _ in tqdm(range(0, n_samples)):
    idx = random.choice(df.index.values)
    seq = df.iloc[idx].Seq
    orig_emb = embeddings[idx]
    all_results.append(compare_reduced_length(seq=seq, orig_emb=orig_emb, n=n))


r = np.stack(all_results)
r.shape

100%|██████████| 1000/1000 [08:55<00:00,  1.87it/s]


(1000, 11)

In [16]:
plt.figure(figsize=(10, 8), dpi=1200)
sns.boxplot(r, whis=(5,95), color="lightblue")
plt.grid(visible=True)
plt.xticks(ticks=[n for n in range(0, 11)], labels=[20*n for n in range(0, 11)])
plt.title(label='DNABERT-2')
plt.ylabel('Cosine Similarity')
plt.xlabel(xlabel='Sequence length difference (in bp)')
plt.ylim(0.97, 1.001)
plt.show()

# DNABERT-S

In [18]:
embeddings =  np.load(Path('../../../Data/Genomes/Greengenes2/2024.09/embeddings/df.2024.09.backbone.full-length.npy'))

In [ ]:
MODEL_ID = "zhihan1996/DNABERT-S"
model, tokenizer = load_model_from_hf(model_name=MODEL_ID)

In [22]:
n_samples = 1000
n = 100
all_results = []

for _ in tqdm(range(0, n_samples)):
    idx = random.choice(df.index.values)
    seq = df.iloc[idx].Seq
    orig_emb = embeddings[idx]
    all_results.append(compare_reduced_length(seq=seq, orig_emb=orig_emb, n=n))


r = np.stack(all_results)
r.shape

100%|██████████| 1000/1000 [09:14<00:00,  1.80it/s]


(1000, 11)

In [23]:
plt.figure(figsize=(10, 8), dpi=1200)
sns.boxplot(r, whis=(5,95), color="lightblue")
plt.grid(visible=True)
plt.xticks(ticks=[n for n in range(0, 11)], labels=[20*n for n in range(0, 11)])
plt.title(label='DNABERT-S')
plt.ylabel('Cosine Similarity')
plt.xlabel(xlabel='Sequence length difference (in bp)')
plt.ylim(0.6, 1.001)
plt.show()